In [4]:
# 导入必要的库
import random
import sys
from datetime import datetime, timedelta

# 添加项目路径，确保能导入你自己的模块
sys.path.append('E:/miniconda/ChineseSuperLeague')

# 导入你自己写的数据库管理器
from database_manager import DatabaseManager

In [5]:
# 创建数据库管理器对象（会自动连接）
db = DatabaseManager()

✅ 数据库连接成功


In [6]:
# 查询所有球队的 ID
cursor = db.connection.cursor()
cursor.execute("SELECT team_id, team_name FROM teams")
teams = cursor.fetchall()
cursor.close()

# 显示球队列表
print("当前数据库中的球队：")
for team in teams:
    print(f"ID: {team['team_id']}, 名称: {team['team_name']}")

当前数据库中的球队：
ID: 2, 名称: 上海海港
ID: 4, 名称: 上海申花
ID: 3, 名称: 北京国安
ID: 1, 名称: 山东泰山


In [8]:
def generate_simulated_matches(num_matches=20):
    """
    生成模拟比赛数据并插入数据库
    :param num_matches: 要生成的比赛数量
    """
    cursor = db.connection.cursor()
    
    # 假设赛季是2024
    season = "2024"
    
    # 起始日期（比如从2024年3月1日开始）
    start_date = datetime(2024, 3, 1)
    
    for i in range(num_matches):
        # 随机选择主队和客队（不能是同一队）
        home_team = random.choice(teams)
        away_team = random.choice([t for t in teams if t['team_id'] != home_team['team_id']])
        
        # 随机生成比分（0~4）
        home_score = random.randint(0, 4)
        away_score = random.randint(0, 4)
        
        # 判断比赛结果
        if home_score > away_score:
            result = 'H'  # 主胜
        elif home_score < away_score:
            result = 'A'  # 客胜
        else:
            result = 'D'  # 平局
        
        # 比赛日期逐轮增加
        match_date = start_date + timedelta(days=i*7)  # 每周一场比赛
        
        # 插入比赛数据
        sql = """
        INSERT INTO matches (season, round_name, match_date, home_team_id, away_team_id, home_score, away_score, result)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql, (
            season,
            f"第{i+1}轮",
            match_date.date(),
            home_team['team_id'],
            away_team['team_id'],
            home_score,
            away_score,
            result
        ))
    
    db.connection.commit()
    cursor.close()
    print(f"✅ 成功插入 {num_matches} 场模拟比赛数据！")

In [9]:
# 生成20场模拟比赛
generate_simulated_matches(20)

✅ 成功插入 20 场模拟比赛数据！


In [10]:
# 查询并显示插入的比赛数据
cursor = db.connection.cursor()
cursor.execute("""
    SELECT m.match_id, m.season, m.round_name, m.match_date,
           ht.team_name AS home_team, m.home_score,
           at.team_name AS away_team, m.away_score, m.result
    FROM matches m
    JOIN teams ht ON m.home_team_id = ht.team_id
    JOIN teams at ON m.away_team_id = at.team_id
    ORDER BY m.match_date
""")
matches = cursor.fetchall()
cursor.close()

# 打印比赛结果
for match in matches:
    print(f"{match['match_date']} | {match['round_name']} | "
          f"{match['home_team']} {match['home_score']} - {match['away_score']} {match['away_team']} | 结果: {match['result']}")

2024-03-01 | 第1轮 | 上海海港 0 - 2 上海申花 | 结果: A
2024-03-01 | 第1轮 | 上海海港 3 - 2 北京国安 | 结果: H
2024-03-08 | 第2轮 | 山东泰山 2 - 0 上海申花 | 结果: H
2024-03-08 | 第2轮 | 北京国安 0 - 1 山东泰山 | 结果: A
2024-03-15 | 第3轮 | 山东泰山 2 - 1 上海申花 | 结果: H
2024-03-15 | 第3轮 | 山东泰山 3 - 3 北京国安 | 结果: D
2024-03-22 | 第4轮 | 山东泰山 4 - 4 北京国安 | 结果: D
2024-03-22 | 第4轮 | 山东泰山 1 - 0 上海申花 | 结果: H
2024-03-29 | 第5轮 | 山东泰山 2 - 3 上海海港 | 结果: A
2024-03-29 | 第5轮 | 上海申花 2 - 2 山东泰山 | 结果: D
2024-04-05 | 第6轮 | 山东泰山 0 - 3 北京国安 | 结果: A
2024-04-05 | 第6轮 | 上海申花 2 - 2 山东泰山 | 结果: D
2024-04-12 | 第7轮 | 北京国安 1 - 1 上海申花 | 结果: D
2024-04-12 | 第7轮 | 上海海港 3 - 2 上海申花 | 结果: H
2024-04-19 | 第8轮 | 上海海港 0 - 3 北京国安 | 结果: A
2024-04-19 | 第8轮 | 上海申花 1 - 4 北京国安 | 结果: A
2024-04-26 | 第9轮 | 上海申花 4 - 1 北京国安 | 结果: H
2024-04-26 | 第9轮 | 山东泰山 1 - 3 北京国安 | 结果: A
2024-05-03 | 第10轮 | 北京国安 3 - 2 上海申花 | 结果: H
2024-05-03 | 第10轮 | 山东泰山 3 - 3 上海申花 | 结果: D
2024-05-10 | 第11轮 | 山东泰山 1 - 3 上海海港 | 结果: A
2024-05-10 | 第11轮 | 山东泰山 2 - 4 北京国安 | 结果: A
2024-05-17 | 第12轮 | 山东泰山 0 - 3 北京国安 | 结果: A
2024-0